## Instruments

LUSID treats instruments as a first class object. They are at the core of much of LUSID's functionality and are required to post transactions and holdings. Read more about how instruments are treated inside LUSID here [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument).

In this notebook we will cover how to work with instruments inside LUSID. We will:

1) Locally import our instrument universe from a CSV file

2) Determine what identifiers we can use for our instruments

3) Upsert our instruments into LUSID

4) List instruments from the instrument universe

5) Test that we can retrieve an instrument by its identifier

6) Add some reference data to our instruments e.g. security type

7) Search for all equity instruments using the security type reference data we added

8) Update the identifier for one of our instruments

9) Delete a handful of instruments 

*First things first, run the cell below to import the libraries and create the LUSID client*

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import uuid
import urllib.parse as urlparse

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('API Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.5.2478.0


### 1) Import our instrument universe

The first thing that we need to do is get the information about our instruments. In this case we are going to import our instrument universe from a CSV file which has been exported from another one of our systems. 

*Run the cell below to import our instruments and show the first 15 securities*

In [2]:
# Import our instruments from a CSV file using Pandas
instruments = pd.read_csv('data/instruments.csv')
# View the first 15 instruments
instruments.head(n=15)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,LN,united_kingdom,BP/,equity,common_stock,NaN
3,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,LN,united_kingdom,BUR,equity,common_stock,NaN
4,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,LN,united_kingdom,EKF,equity,common_stock,NaN
5,Glencore_LondonStockEx_GLEN,imd_34534555,GBP,JE00B4T3BW64,BBG001MM1KV4,LN,united_kingdom,GLEN,equity,common_stock,NaN
6,JustEat_LondonStockEx_JE,imd_35436366,GBP,GB00BKX5CN86,BBG0065YWM39,LN,united_kingdom,JE/,equity,common_stock,NaN
7,Kingfisher_LondonStockEx_KGF,imd_34535552,GBP,GB0033195214,BBG000BKH1W6,LN,united_kingdom,KGF,equity,common_stock,NaN
8,MicroFocus_LondonStockEx_MCRO,imd_34567338,GBP,GB00BD8YWM01,BBG000G4KKD2,LN,united_kingdom,MCRO,equity,common_stock,NaN
9,RELXGroup_LondonStockEx_REL,imd_43532542,GBP,GB00B2B0DG97,BBG000D03XD4,LN,united_kingdom,REL,equity,common_stock,NaN


Here we can see that our instrument universe contains a mix of equities and bonds. Each instrument has a number of identifiers attached to it as well as some basic reference information. 

### 2) Get allowable instrument identifiers

Great now we have our instrument information we want to add our instruments to LUSID.

To add each instrument to LUSID we must have at least one unique identifier for it. 

This is because LUSID uses a bi-temporal data store, which means that no instrument is ever truly deleted. There are thus restrictions on what can and can't be used as a unique identifier. This ensures that there are no collisions between instruments which would arise from using an identifier which may be re-purposed. Read more about LUSID's bi-temporal data store here [LUSID Knowledge Base: Bi-Temporal Data Store](https://support.lusid.com/what-is-bi-temporal-data).

We can call LUSID to find out what the allowable types of instrument indentifiers are. Read more about identifier schemes in LUSID here [LUSID Knowledge Base: Identifier Schemes](https://support.lusid.com/which-instrument-identifier-schemes-should-i-use-with-lusid). 

For further usage of the get instrument identifiers API call refer to the [LUSID API Docs: Get Allowable Instrument Identifiers](https://docs.lusid.com/#operation/GetInstrumentIdentifiers).

*Run the cell below to get our allowable instrument identifiers*

In [4]:
# Call LUSID to get our allowable instrument indentifiers
response = client.instruments.get_instrument_identifiers()

# Extract the unique identifiers from the response and print them
prettyprint.get_identifiers(response, unique=True)

    Identifier Name : LusidInstrumentId
    Is Unique Identifier : True
    Identifier Property Key Value : Instrument/default/LusidInstrumentId


    Identifier Name : ClientInternal
    Is Unique Identifier : True
    Identifier Property Key Value : Instrument/default/ClientInternal


    Identifier Name : QuotePermId
    Is Unique Identifier : True
    Identifier Property Key Value : Instrument/default/QuotePermId


    Identifier Name : Figi
    Is Unique Identifier : True
    Identifier Property Key Value : Instrument/default/Figi




These are the unique identifiers that we can use when adding our instruments to LUSID. We must provide at least one of them for each instrument. 

We can also find the available non-unique identifiers.

*Run the cell below to see the non-unique identifiers that are available*

In [5]:
# Extract the non-unique identifiers from the response and print them
prettyprint.get_identifiers(response, unique=False)

    Identifier Name : ShareClassFigi
    Is Unique Identifier : False
    Identifier Property Key Value : Instrument/default/ShareClassFigi


    Identifier Name : Ticker
    Is Unique Identifier : False
    Identifier Property Key Value : Instrument/default/Ticker


    Identifier Name : Isin
    Is Unique Identifier : False
    Identifier Property Key Value : Instrument/default/Isin


    Identifier Name : Sedol
    Is Unique Identifier : False
    Identifier Property Key Value : Instrument/default/Sedol


    Identifier Name : Cusip
    Is Unique Identifier : False
    Identifier Property Key Value : Instrument/default/Cusip


    Identifier Name : RIC
    Is Unique Identifier : False
    Identifier Property Key Value : Instrument/default/RIC


    Identifier Name : Wertpapier
    Is Unique Identifier : False
    Identifier Property Key Value : Instrument/default/Wertpapier


    Identifier Name : CompositeFigi
    Is Unique Identifier : False
    Identifier Property Key Value : Ins

### 3) Upsert instruments

Now that we know which instrument identifiers we can use, we can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument).

As part of this definition we will also attach two alias identifiers to our instruments. Read more about alias identifiers here [LUSID Knowledge Base: What is an Alias Identifier?](https://support.lusid.com/alias-identifiers).

We use an upsert method to add instrument definitions to the instrument universe. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert our instruments into LUSID*

In [6]:
# Initialise dictionary to hold our instrument definitions
definitions = {}

# Iterate over each row in our instrument universe
for row, instrument in instruments.iterrows():
    
    # Specify your identifier columns
    identifier_columns = [
        ('isin', 'Isin'), 
        ('figi', 'Figi'), 
        ('ticker', 'Ticker'),
        ('client_internal', 'ClientInternal')
    ]
    
    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier[1]] = models.InstrumentIdValue(
            value=instrument[identifier[0]])
    
    # Create our definitions
    definitions[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers)
    
# Call the LUSID API to upsert our instrument definitions
response = client.instruments.upsert_instruments(
    requests=definitions)

# Print each the definition of each instrument returned in the response
for figi, instrument in response.values.items():
    print('Figi: ' + figi + '\n',
          'Instrument Name: ' + instrument.name + '\n', 
          'LUSID Instrument ID: ' + instrument.lusid_instrument_id + '\n'+'\n')

Figi: USTreasury_2.00_2021
 Instrument Name: USTreasury_2.00_2021
 LUSID Instrument ID: LUID_A7TTZPMT


Figi: Glencore_LondonStockEx_GLEN
 Instrument Name: Glencore_LondonStockEx_GLEN
 LUSID Instrument ID: LUID_SOUT97YW


Figi: MicroFocus_LondonStockEx_MCRO
 Instrument Name: MicroFocus_LondonStockEx_MCRO
 LUSID Instrument ID: LUID_ZLD4WV9N


Figi: Kingfisher_LondonStockEx_KGF
 Instrument Name: Kingfisher_LondonStockEx_KGF
 LUSID Instrument ID: LUID_V61U9ZEP


Figi: UKGiltTreasury_3.75_2021
 Instrument Name: UKGiltTreasury_3.75_2021
 LUSID Instrument ID: LUID_7BCZVRZ6


Figi: Whitebread_LondonStockEx_WTB
 Instrument Name: Whitebread_LondonStockEx_WTB
 LUSID Instrument ID: LUID_M6MDIWQB


Figi: UKGiltTreasury_4.5_2034
 Instrument Name: UKGiltTreasury_4.5_2034
 LUSID Instrument ID: LUID_CIKCBMRE


Figi: UKGiltTreasury_2.0_2025
 Instrument Name: UKGiltTreasury_2.0_2025
 LUSID Instrument ID: LUID_X6YTNSKS


Figi: RELXGroup_LondonStockEx_REL
 Instrument Name: RELXGroup_LondonStockEx_REL
 LUS

We have added our instruments to LUSID! We can now use these instruments to book trades and set holdings.

As you can see every instrument that is upserted into LUSID is given a LUSID unique instrument Identifier. This is a globally unique identifier for the instrument and is used in some calls to LUSID. Read more about LUSID Unique Instrument Identifiers here [LUSID Knoweldge Base: LUSID Instrument Identifiers](https://support.lusid.com/what-is-a-lusid-unique-identifier-luid).

### 4) List our Instruments

We can list instruments from our instrument universe. Note that if some instruments have already been added to the universe the instruments we just upserted may not be returned in the first few pages of results.

For further usage of the list instruments API call refer to the [LUSID API Docs: List Instruments](https://docs.lusid.com/#operation/ListInstruments).

*Run the cell below to list 10 instruments from our instrument universe*

In [5]:
# Call the LUSID API to list our instruments, limit results to 10 per page
response = client.instruments.list_instruments(
    limit=10
)

# Pretty print the result
prettyprint.list_instruments_response(response)

Previous Page Link: N/A

Next Page Link: https://api.lusid.com/api/instruments/?effectiveAt=2019-04-05T11%3A48%3A46.3418240%2B00%3A00&asAt=2019-04-05T11%3A48%3A44.1282800%2B00%3A00&limit=10&filter=State%20eq%20%27Active%27&page=JbF2OgCAizapvLnWCPDI5Ke8udYITFVJRF8wMDFWSEdVQg%3D%3D

Instrument Name: AGM    2.220 01/18/22 '19 MTN
LUSID Instrument ID: LUID_000B882J
QuotePermId: 46642662707


Instrument Name: FIRST REPUB DS REP NON PER SRS D PRF
LUSID Instrument ID: LUID_000GD8SE
QuotePermId: 21522749249


Instrument Name: HUGO BOSS ADR
LUSID Instrument ID: LUID_000VXEQS
QuotePermId: 21517696396


Instrument Name: CMCSA  5.150 04/30/20
LUSID Instrument ID: LUID_000XMNBJ
QuotePermId: 34487275680


Instrument Name: GLADSTONE INVESTMENT ORD
LUSID Instrument ID: LUID_00119TZY
QuotePermId: 21475115857


Instrument Name: BANDWIDTH CL A ORD
LUSID Instrument ID: LUID_00130SAL
QuotePermId: 25727416435


Instrument Name: WEST JAPAN RAILWAY ORD
LUSID Instrument ID: LUID_0014FT88
QuotePermId: 558354177

We can page through our results to the next page using the page token at the end of the next page URL.

*Run the cell below to page through to the next page of results*

In [6]:
# Parse our page token from the link URL
for link in response.links:
    if link.relation == 'NextPage':
        parsed_url = urlparse.urlparse(link.href)
        page_token = urlparse.parse_qs(parsed_url.query)['page'][0]
        
# Call the LUSID API to list our instruments
response = client.instruments.list_instruments(
    limit=10,
    page=page_token
)

# Pretty print the result
prettyprint.list_instruments_response(response)

Previous Page Link: https://api.lusid.com/api/instruments/?effectiveAt=2019-04-05T11%3A48%3A46.3418240%2B00%3A00&asAt=2019-04-05T11%3A48%3A44.1282800%2B00%3A00&limit=10&filter=State%20eq%20%27Active%27&page=HSUhxwGAizapvLnWCPDI5Ke8udYITFVJRF8wMDE5QUJRWA%3D%3D

Next Page Link: https://api.lusid.com/api/instruments/?effectiveAt=2019-04-05T11%3A48%3A46.3418240%2B00%3A00&asAt=2019-04-05T11%3A48%3A44.1282800%2B00%3A00&limit=10&filter=State%20eq%20%27Active%27&page=JyeFSgCAizapvLnWCPDI5Ke8udYITFVJRF8wMDNMSlJNMA%3D%3D

Instrument Name: AVENUE THERAPEUTICS ORD
LUSID Instrument ID: LUID_001VHGUB
QuotePermId: 25727436442


Instrument Name: EYEPOINT PHARMACEUTICALS ORD
LUSID Instrument ID: LUID_001WCEMY
QuotePermId: 21475114825


Instrument Name: HAMILTON BANCORP ORD
LUSID Instrument ID: LUID_0024FCWX
QuotePermId: 21506482326


Instrument Name: DATADOT TECH ORD
LUSID Instrument ID: LUID_0029POY6
QuotePermId: 55840307918


Instrument Name: DYNAMIC ISHARES ACTV CAN DIV UNT ETF
LUSID Instrument ID: 

We can also go back through our results to the previous page using the page token at the end of the previous page URL.

*Run the cell below to page through to the previous page of results*

In [7]:
# Parse our page token from the link URL
for link in response.links:
    if link.relation == 'PreviousPage':
        parsed_url = urlparse.urlparse(link.href)
        page_token = urlparse.parse_qs(parsed_url.query)['page'][0]

# Call the LUSID API to list our instruments
response = client.instruments.list_instruments(
    limit=10,
    page=page_token
)

# Pretty print the result
prettyprint.list_instruments_response(response)

Previous Page Link: N/A

Next Page Link: https://api.lusid.com/api/instruments/?effectiveAt=2019-04-05T11%3A48%3A46.3418240%2B00%3A00&asAt=2019-04-05T11%3A48%3A44.1282800%2B00%3A00&limit=10&filter=State%20eq%20%27Active%27&page=JbF2OgCAizapvLnWCPDI5Ke8udYITFVJRF8wMDFWSEdVQg%3D%3D

Instrument Name: AGM    2.220 01/18/22 '19 MTN
LUSID Instrument ID: LUID_000B882J
QuotePermId: 46642662707


Instrument Name: FIRST REPUB DS REP NON PER SRS D PRF
LUSID Instrument ID: LUID_000GD8SE
QuotePermId: 21522749249


Instrument Name: HUGO BOSS ADR
LUSID Instrument ID: LUID_000VXEQS
QuotePermId: 21517696396


Instrument Name: CMCSA  5.150 04/30/20
LUSID Instrument ID: LUID_000XMNBJ
QuotePermId: 34487275680


Instrument Name: GLADSTONE INVESTMENT ORD
LUSID Instrument ID: LUID_00119TZY
QuotePermId: 21475115857


Instrument Name: BANDWIDTH CL A ORD
LUSID Instrument ID: LUID_00130SAL
QuotePermId: 25727416435


Instrument Name: WEST JAPAN RAILWAY ORD
LUSID Instrument ID: LUID_0014FT88
QuotePermId: 558354177

### 5) Retrieve an Instrument by its Identifier

In addition to listing all instruments we can also retrieve an instrument individually by searching for it by one of its unique identifiers. 

Here we can look up one of our instruments by its Financial Instrument Global Identifier (FIGI). 

For further usage of the get instrument API call refer to the [LUSID API Docs: Get Instrument Defintion](https://docs.lusid.com/#operation/GetInstrument).

*Run the cell below to retrieve our instrument's definition by its FIGI*

In [8]:
# Call the LUSID API to retrieve an instrument by its FIGI
response = client.instruments.get_instrument(
    identifier_type='Figi',
    identifier='BBG000C05BD1')

# Print the response with our instrument definition
print('Instrument Name: ' + response.name + '\n', 
      'FIGI: ' + response.identifiers['Figi'] + '\n',
      'ClientInternal: ' + response.identifiers['ClientInternal'] + '\n',
      'LUSID Instrument ID: ' + response.lusid_instrument_id)

Instrument Name: BP_LondonStockEx_BP
 FIGI: BBG000C05BD1
 ClientInternal: imd_43535553
 LUSID Instrument ID: LUID_XTL9U9LB


As we gave each of our instruments two unique identifiers, i.e. a FIGI and a ClientInternal, we can also look up the same instrument by its ClientInternal identifier.

*Run the cell below to retrieve our instrument definition by its ClientInternal identifier*

In [9]:
# Call the LUSID API to retrieve an instrument by its ClientInternal ID
response = client.instruments.get_instrument(
    identifier_type='ClientInternal',
    identifier='imd_43535553')

# Print the response with our instrument
print('Instrument Name: ' + response.name + '\n', 
      'FIGI: ' + response.identifiers['Figi'] + '\n',
      'ClientInternal: ' + response.identifiers['ClientInternal'] + '\n',
      'LUSID Instrument ID: ' + response.lusid_instrument_id)

Instrument Name: BP_LondonStockEx_BP
 FIGI: BBG000C05BD1
 ClientInternal: imd_43535553
 LUSID Instrument ID: LUID_XTL9U9LB


We can also retrieve multiple instruments by passing multiple values of a given identifier into the get instruments method. For further usage of the get instruments API call refer to the [LUSID API Docs: Get Instrument Defintion](https://docs.lusid.com/#operation/GetInstruments).

*Run the cell below to retrieve multiple instrument definitions by their FIGI*

In [10]:
# Call the LUSID API to retrieve all instruments with a Figi in the provided list
response = client.instruments.get_instruments(
    identifier_type='Figi',
    identifiers=['BBG000C05BD1', 'BBG000DQQNJ8'])

# Print the response with our instrument defintions
for figi, instrument in response.values.items():
    print('Instrument Name: ' + instrument.name + '\n', 
          'FIGI: ' + figi + '\n',
          'ClientInternal: ' + instrument.identifiers['ClientInternal'] + '\n',
          'LUSID Instrument ID: ' + instrument.lusid_instrument_id + '\n')

Instrument Name: BP_LondonStockEx_BP
 FIGI: BBG000C05BD1
 ClientInternal: imd_43535553
 LUSID Instrument ID: LUID_XTL9U9LB

Instrument Name: USTreasury_6.875_2025
 FIGI: BBG000DQQNJ8
 ClientInternal: imd_34534539
 LUSID Instrument ID: LUID_4FGGTYKN



### 6) Upsert instrument properties

In addition to our unique identifier and aliases we can also add properties to our instruments. 

To define a property we need to create a property definition and then call LUSID to create that property. Read more about property defintions here [LUSID Knowledge Base: Property Definitions](https://support.lusid.com/what-is-a-property-definition). 

Note that every property is defined by a data type which is referenced by a data type id. Read more about data types here [LUSID Knowledge Base: Data Types](https://support.lusid.com/what-is-a-data-type).

Let us create a property to hold the asset class of our instrument.

For further usage of the create property definition API call refer to the [LUSID API Docs: Create Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition).

*Run the cell below to create our security type property*

In [11]:
# Define the scope that we will use for our properties
properties_scope = 'custom_properties_{}'.format(str(uuid.uuid4())[:4])

# Create the request for our new property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Instrument',
    scope=properties_scope,
    code='asset_class',
    value_required=True,
    display_name='asset_class',
    data_type_id=models.ResourceId(
        scope='default', 
        code='string'))

# Call LUSID to create our new property
property_response = client.property_definitions.create_property_definition(
    definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
asset_class_property_key = property_response.key

# Print our property key
print (asset_class_property_key)

Instrument/custom_properties_6b6e/asset_class


As you can see from the call above every property is identified by its key. Read more about properties and their keys here [LUSID Knowledge Base: Properties](https://support.lusid.com/what-is-a-property-definition).

We can now add this property to each of our instruments.

For further usage of the upsert instrument properties API call refer to the [LUSID API Docs: Upserting Instrument Properties](https://docs.lusid.com/#operation/UpsertInstruments)

*Run the cell below to set our asset class property for each of our instruments*

Now that we have created our property we can set it across our instrument universe.

*Run the cell below to set our asset class property for each of our instruments*

In [12]:
requests = []

for row in instruments.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property = models.InstrumentProperty(
        key=asset_class_property_key,
        value=models.PropertyValue(
            label_value=instrument['market_sector'],
            effective_from=None))
    
    # Call the LUSID API to get our LUSID Instrument ID for this instrument
    response = client.instruments.get_instrument(
        identifier_type='Figi',
        identifier=instrument['figi'])
    
    # Set the LUSID Instrument ID
    luid = response.lusid_instrument_id
    
    # Build our request to set our property
    requests.append(
        models.UpsertInstrumentPropertyRequest(
            lusid_instrument_id=luid,
            properties=[asset_class_property],
            deleted_properties=None))

# Call the LUSID API to add our property across all instruments
response = client.instruments.upsert_instruments_properties(
    instrument_properties=requests)

# Print our response

print ('Instrument properties upserted at {}'.format(
    response.as_at_date))

Instrument properties upserted at 2019-04-05 11:48:55.533822+00:00


We can now call LUSID to retrieve an instrument definition for one of our instruments with our user defined security type property attached. We do this by speicfying the property key in the instrument_property_keys argument of the method. 

*Run the cell below to call LUSID and retrieve our instrument*

In [13]:
# Call LUSID to get the ISIN and security type for one of our instruments
response = client.instruments.get_instrument(
    identifier_type='Figi',
    identifier='BBG000C05BD1',
    instrument_property_keys=[
        asset_class_property_key
    ])

# Print the response
print('Instrument Name: ' + response.name + '\n', 
      'Figi: ' + response.identifiers['Figi'] + '\n',
      'ClientInternal: ' + response.identifiers['ClientInternal'] + '\n',
      'Isin: ' + response.identifiers['Isin'] + '\n',
      'Ticker: ' + response.identifiers['Ticker'] + '\n',
      'Lusid Instrument ID: ' + response.lusid_instrument_id + '\n',
      'Asset Class: ' + response.properties[0].value + '\n')

Instrument Name: BP_LondonStockEx_BP
 Figi: BBG000C05BD1
 ClientInternal: imd_43535553
 Lusid Instrument ID: LUID_XTL9U9LB
 Asset Class: equity



Note that we can also retrieve our instrument's alias identifiers this way.

*Run the cell below to call LUSID and retrieve our instrument with its aliases*

In [14]:
# Call LUSID to get the ISIN and security type for one of our instruments
response = client.instruments.get_instrument(
    identifier_type='Figi',
    identifier='BBG000C05BD1',
    instrument_property_keys=[
        asset_class_property_key,
        'Instrument/default/Ticker',
        'Instrument/default/Isin'
    ])

# Print the response
print('Instrument Name: ' + response.name + '\n', 
      'Figi: ' + response.identifiers['Figi'] + '\n',
      'ClientInternal: ' + response.identifiers['ClientInternal'] + '\n',
      'Lusid Instrument ID: ' + response.lusid_instrument_id + '\n',
      'Asset Class: ' + response.properties[2].value + '\n',
      'Ticker: ' + response.properties[1].value + '\n',
      'ISIN: ' + response.properties[0].value + '\n')

Instrument Name: BP_LondonStockEx_BP
 Figi: BBG000C05BD1
 ClientInternal: imd_43535553
 Lusid Instrument ID: LUID_XTL9U9LB
 Asset Class: equity
 Ticker: BP/
 ISIN: GB0007980591



We can also delete a property using the same upsert properties endpoint. We do this by specifying the property key that we'd like to delete. 

*Run the cell below to delete the asset class property for one of our instruments*

In [15]:
delete_asset_class_property = models.DeleteInstrumentPropertyRequest(
    instrument_property_key=asset_class_property_key,
    effective_from=datetime.now(pytz.UTC))

# Call the LUSID API to get our LUSID Instrument ID for this instrument
response = client.instruments.get_instrument(
    identifier_type='Figi',
    identifier="BBG000C05BD1")

# Set the LUSID Instrument ID
luid = response.lusid_instrument_id

# Build our request to set our property
request = models.UpsertInstrumentPropertyRequest(
              lusid_instrument_id=luid,
              deleted_properties=[
                  delete_asset_class_property
              ])

# Call the LUSID API to add our property across all instruments
response = client.instruments.upsert_instruments_properties(
    instrument_properties=[request])

# Print our response
print ('Properties deleted at {}'.format(
    response.as_at_date))

Properties deleted at 2019-04-05 11:48:58.033023+00:00


### 7) Search instruments

Now that we have set some properties on our instruments, we can now go ahead and search our instrument universe that we've just created using these properties.

In this case let's search for all equity instruments using our asset class property.

For further usage of the instruments search API call refer to the [LUSID API Docs: Search Instruments](https://docs.lusid.com/#operation/InstrumentsSearch)

*Run the cell below to search for all our equity securities*

In [16]:
# Build our serach request
search_request_internal = models.InstrumentSearchProperty(
    key=asset_class_property_key,
    value='equity')

# Call LUSID to search for instruments
response = client.search.instruments_search(
    symbols=[search_request_internal],
    mastered_only=True)

# Print the instrument definitions from our response
for instrument in response[0].mastered_instruments:
    print('Instrument Name: ' + instrument.name + '\n', 
          'Figi: ' + instrument.identifiers['Figi'].value + '\n',
          'ClientInternal: ' + instrument.identifiers['ClientInternal'].value + '\n',
          'LUSID Instrument ID: ' + instrument.identifiers['LusidInstrumentId'].value +'\n')

Instrument Name: Amazon_Nasdaq_AMZN
 Figi: BBG000BVPXP1
 ClientInternal: imd_34634534
 LUSID Instrument ID: LUID_2HT2UEN4

Instrument Name: Apple_Nasdaq_AAPL
 Figi: BBG000B9XVV8
 ClientInternal: imd_35345345
 LUSID Instrument ID: LUID_5DJ0Y5VF

Instrument Name: BurfordCapital_LondonStockEx_BUR
 Figi: BBG000PN88Q7
 ClientInternal: imd_43534356
 LUSID Instrument ID: LUID_UUXNQOCF

Instrument Name: EKFDiagnostics_LondonStockEx_EKF
 Figi: BBG000BVNBN3
 ClientInternal: imd_34535355
 LUSID Instrument ID: LUID_S783ULAM

Instrument Name: Glencore_LondonStockEx_GLEN
 Figi: BBG001MM1KV4
 ClientInternal: imd_34534555
 LUSID Instrument ID: LUID_33YI8NH5

Instrument Name: JustEat_LondonStockEx_JE
 Figi: BBG0065YWM39
 ClientInternal: imd_35436366
 LUSID Instrument ID: LUID_UNXUUKCV

Instrument Name: Kingfisher_LondonStockEx_KGF
 Figi: BBG000BKH1W6
 ClientInternal: imd_34535552
 LUSID Instrument ID: LUID_V1JN41X3

Instrument Name: MicroFocus_LondonStockEx_MCRO
 Figi: BBG000G4KKD2
 ClientInternal: imd

Using search we can also search for instruments that are not currently in the instrument universe inside LUSID. 

*Run the cell below to search for an instrument that is not in our instrument universe*

In [17]:
search_request_external = models.InstrumentSearchProperty(
    key='Instrument/default/Figi',
    value='BBG000BD5TW4')

response = client.search.instruments_search(
    symbols=[search_request_external],
    mastered_only=False)

for instrument in response[0].external_instruments:
    print ('Instrument Name: ' + instrument.name + '\n', 
          'Figi: ' + instrument.identifiers['Figi'].value)
    for _property in instrument.properties:
        print (_property.key, '-', _property.value.label_value)
    print ('\n')

Instrument Name: BAE SYSTEMS PLC
 Figi: BBG000BD5TW4
Instrument/default/CompositeFigi - BBG000BD5T18
Instrument/default/ShareClassFigi - BBG001S615R1
Instrument/default/Ticker - BA/




We can also chain our search requests together. When we use multiple search requests LUSID uses OR logic.

*Run the cell below to use both our search requests together*

In [18]:
response = client.search.instruments_search(
    symbols=[search_request_internal, search_request_external],
    mastered_only=False)

for instrument in response[0].mastered_instruments:
    print('Instrument Name: ' + instrument.name + '\n', 
          'Figi: ' + instrument.identifiers['Figi'].value + '\n',
          'ClientInternal: ' + instrument.identifiers['ClientInternal'].value + '\n',
          'LUSID Instrument ID: ' + instrument.identifiers['LusidInstrumentId'].value +'\n')

for instrument in response[1].external_instruments:
    print ('Instrument Name: ' + instrument.name + '\n', 
          'Figi: ' + instrument.identifiers['Figi'].value)
    for _property in instrument.properties:
        print (_property.key, '-', _property.value.label_value)
    print ('\n')

Instrument Name: Amazon_Nasdaq_AMZN
 Figi: BBG000BVPXP1
 ClientInternal: imd_34634534
 LUSID Instrument ID: LUID_2HT2UEN4

Instrument Name: Apple_Nasdaq_AAPL
 Figi: BBG000B9XVV8
 ClientInternal: imd_35345345
 LUSID Instrument ID: LUID_5DJ0Y5VF

Instrument Name: BurfordCapital_LondonStockEx_BUR
 Figi: BBG000PN88Q7
 ClientInternal: imd_43534356
 LUSID Instrument ID: LUID_UUXNQOCF

Instrument Name: EKFDiagnostics_LondonStockEx_EKF
 Figi: BBG000BVNBN3
 ClientInternal: imd_34535355
 LUSID Instrument ID: LUID_S783ULAM

Instrument Name: Glencore_LondonStockEx_GLEN
 Figi: BBG001MM1KV4
 ClientInternal: imd_34534555
 LUSID Instrument ID: LUID_33YI8NH5

Instrument Name: JustEat_LondonStockEx_JE
 Figi: BBG0065YWM39
 ClientInternal: imd_35436366
 LUSID Instrument ID: LUID_UNXUUKCV

Instrument Name: Kingfisher_LondonStockEx_KGF
 Figi: BBG000BKH1W6
 ClientInternal: imd_34535552
 LUSID Instrument ID: LUID_V1JN41X3

Instrument Name: MicroFocus_LondonStockEx_MCRO
 Figi: BBG000G4KKD2
 ClientInternal: imd

### 8) Update instrument identifier

We can also update an instruments identifiers. For further usage of the update instrument identifier API call refer to the [LUSID API Docs: Update an Instrument Identifier](https://docs.lusid.com/#operation/UpdateInstrumentIdentifier).

*Run the cell below to update the Client Internal Id for one of our instruments*

In [19]:
request = models.UpdateInstrumentIdentifierRequest(
    type='ClientInternal',
    value='imd_43535554',
    effective_at=None)

response = client.instruments.update_instrument_identifier(
    identifier_type='Figi',
    identifier='BBG000C05BD1',
    request=request)

# Print the response showing the updated ClientInternal ID
print('Instrument Name: ' + response.name + '\n', 
      'Figi: ' + response.identifiers['Figi'] + '\n',
      'ClientInternal: ' + response.identifiers['ClientInternal'] + '\n',
      'Isin: ' + response.identifiers['Isin']+ '\n',
      'Ticker: ' + response.identifiers['Ticker'] + '\n',
      'Lusid Instrument ID: ' + response.lusid_instrument_id + '\n')

Instrument Name: BP_LondonStockEx_BP
 Figi: BBG000C05BD1
 ClientInternal: imd_43535554
 Lusid Instrument ID: LUID_XTL9U9LB



### 9) Delete instruments

In addition to upserting instruments we can also delete them. When we delete an instrument it becomes unvailable for us to use from the time that we delete it onwards. If we want to look back in time at our instruments or transactions in those instruments we will be able to find them, they are not permanently deleted from the system.

For further usage of the delete instrument API call refer to the [LUSID API Docs: Delete Instrument](https://docs.lusid.com/#operation/DeleteInstrument).


*Run the cell below to delete our instrument universe*

In [20]:
for figi in instruments.loc[:, 'figi'].values:

    response = client.instruments.delete_instrument(
        identifier_type='Figi',
        identifier=figi)
    
    print ('Instrument with FIGI {} deleted'.format(figi))

Instrument with FIGI BBG000BVPXP1 deleted
Instrument with FIGI BBG000B9XVV8 deleted
Instrument with FIGI BBG000C05BD1 deleted
Instrument with FIGI BBG000PN88Q7 deleted
Instrument with FIGI BBG000BVNBN3 deleted
Instrument with FIGI BBG001MM1KV4 deleted
Instrument with FIGI BBG0065YWM39 deleted
Instrument with FIGI BBG000BKH1W6 deleted
Instrument with FIGI BBG000G4KKD2 deleted
Instrument with FIGI BBG000D03XD4 deleted
Instrument with FIGI BBG000BN0PP3 deleted
Instrument with FIGI BBG000BF46Y8 deleted
Instrument with FIGI BBG0088JSC32 deleted
Instrument with FIGI BBG006N6HZM7 deleted
Instrument with FIGI BBG001KKJLR4 deleted
Instrument with FIGI BBG0000D14P3 deleted
Instrument with FIGI BBG00FN3B5K8 deleted
Instrument with FIGI BBG000DQQNJ8 deleted
Instrument with FIGI BBG000BF6B57 deleted
Instrument with FIGI BBG000BRVH05 deleted
